In [1]:
import pandas as pd

# Cargar el archivo CSV 'insert_soluciones'
df = pd.read_csv('./sql/insert_soluciones.csv')
df.sort_values(by='id')

# Mostrar las primeras filas para verificar la carga
print(df.head())


   id                                               name  \
0   1          Acompañamiento en innovación - Innobasque   
1   2  Orientación personalizada sobre oportunidades ...   
2   3  Búsqueda de socios internacionales en proyecto...   
3   4    Formación en proyectos internacionales de I+D+i   
4   5  Transferencia de tecnologías, patentes y know-...   

                                    long_description  \
0  - Escucha y orientación básica: Diagnóstico de...   
1  Si necesitas ayuda para presentar una propuest...   
2  Como miembro de varias redes internacionales (...   
3  Diferentes cursos en función del grado de cono...   
4  Incorpora las mejores prácticas de la industri...   

                                   short_description  \
0  Servicio de acompañamiento en materia de innov...   
1  Te acompañamos a lo largo de todo el proceso d...   
2  Asistencia a entidades vascas que buscan socio...   
3  Programa formativo, prioritariamente dirigido ...   
4  Transferencia de te

In [2]:
# Analizar si todos los campos del CSV son UTF-8 y detectar problemas de encoding

def check_utf8_encoding(df):
    problematic_rows = []
    for idx, row in df.iterrows():
        for col in df.columns:
            value = row[col]
            # Solo analizar si es string
            if isinstance(value, str):
                try:
                    value.encode('utf-8')
                except UnicodeEncodeError:
                    problematic_rows.append((idx, col, value))
    if problematic_rows:
        print("Problemas de encoding detectados en las siguientes filas y columnas:")
        for idx, col, value in problematic_rows:
            print(f"Fila {idx}, columna '{col}': valor problemático -> {repr(value)}")
    else:
        print("Todos los campos string del CSV son compatibles con UTF-8.")

check_utf8_encoding(df)


Todos los campos string del CSV son compatibles con UTF-8.


In [3]:
import psycopg2
import os

DB_NAME = os.getenv('POSTGRES_DB')
USER = os.getenv('POSTGRES_USER')
PASSWORD = os.getenv('POSTGRES_PASSWORD')

# Parámetros de conexión para PostgreSQL (ajusta según tu entorno)
db_params = {
    'host': 'localhost',
    'port': 5432,
    'dbname': DB_NAME,      # Cambia por el nombre real de tu base de datos PostgreSQL
    'user': USER,            # Cambia por tu usuario de PostgreSQL
    'password': PASSWORD      # Cambia por tu contraseña de PostgreSQL
}

In [4]:
# Cargar datos manejando valores NULL correctamente
import psycopg2
import numpy as np

# Obtener las primeras 10 filas
df_sample = df

print(f"Cargando {len(df_sample)} filas en la tabla soluciones_ayuda_formacion_es...")

try:
    # Conectar a la base de datos
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    
    # Preparar los datos para insertar
    columns = df_sample.columns.tolist()
    
    # 🔧 SOLUCIÓN: Convertir NaN a None para que PostgreSQL los trate como NULL
    df_clean = df_sample.copy()
    df_clean = df_clean.replace({np.nan: None})
    values = df_clean.values.tolist()
    
    print("Valores NaN convertidos a None para PostgreSQL")
    
    # Crear la query de INSERT
    placeholders = ', '.join(['%s'] * len(columns))
    query = f"""
        INSERT INTO soluciones_ayuda_formacion_es ({', '.join(columns)})
        VALUES ({placeholders})
    """
    
    # Insertar los datos
    cursor.executemany(query, values)
    
    # Confirmar los cambios
    conn.commit()
    
    print(f"Se insertaron {len(df_sample)} filas correctamente")
    
    # Verificar que se insertaron
    cursor.execute("SELECT COUNT(*) FROM soluciones_ayuda_formacion_es")
    count = cursor.fetchone()[0]
    print(f"Total de registros en la tabla: {count}")
    
except Exception as e:
    print(f"Error al insertar datos: {e}")
    if 'conn' in locals():
        conn.rollback()
finally:
    if 'conn' in locals():
        cursor.close()
        conn.close()


Cargando 1873 filas en la tabla soluciones_ayuda_formacion_es...
Valores NaN convertidos a None para PostgreSQL
Se insertaron 1873 filas correctamente
Total de registros en la tabla: 1873


In [5]:
# Verificar que los datos se cargaron correctamente
try:
    conn = psycopg2.connect(**db_params)
    
    # Consulta para ver las primeras 5 filas insertadas
    query = "SELECT * FROM soluciones_ayuda_formacion_es LIMIT 5;"
    df_verificacion = pd.read_sql_query(query, conn)
    
    print("🔍 Verificación de datos cargados:")
    print(f"📊 Total de registros: {len(df_verificacion)}")
    print("\n📋 Primeras 5 filas:")
    display(df_verificacion)
    
    conn.close()
    
except Exception as e:
    print(f"Error al verificar datos: {e}")


🔍 Verificación de datos cargados:
📊 Total de registros: 5

📋 Primeras 5 filas:


C:\Users\JOSEBA wac\AppData\Local\Temp\ipykernel_2364\437705358.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verificacion = pd.read_sql_query(query, conn)


,id,name,long_description,short_description,web,req_size,req_tag_title,req_tag_description,amount_title_1,amount_description_1,...,nombres_areas_boletin,aplicacion_geo,size,sector_name,sector_specialization_name,soport_types,expenses,created_at,updated_at,deleted_at
0,1,Acompañamiento en innovación - Innobasque,- Escucha y orientación básica: Diagnóstico de...,Servicio de acompañamiento en materia de innov...,https://profilak.innobasque.eus/escucha/registro,None,None,None,None,None,...,None,Euskadi,+250,Multisectorial,Multisectorial,Acompañamiento para innovar,Gratuito,2019-05-14 15:36:32,2025-02-17 17:30:54,None
1,2,Orientación personalizada sobre oportunidades ...,Si necesitas ayuda para presentar una propuest...,Te acompañamos a lo largo de todo el proceso d...,https://www.eenbasque.net/,None,None,None,None,None,...,None,Euskadi,+250,Multisectorial,Multisectorial,Asesoramiento experto,Gratuito,2019-05-15 07:48:04,2025-03-17 16:25:40,None
2,3,Búsqueda de socios internacionales en proyecto...,Como miembro de varias redes internacionales (...,Asistencia a entidades vascas que buscan socio...,https://een.ec.europa.eu/partnering-opportunities,None,None,None,None,None,...,None,Euskadi,+250,Multisectorial,Multisectorial,Asesoramiento experto,Gratuito,2019-05-15 07:53:40,2025-04-01 21:24:08,None
3,4,Formación en proyectos internacionales de I+D+i,Diferentes cursos en función del grado de cono...,"Programa formativo, prioritariamente dirigido ...",https://www.innobasque.eus/te-ayudamos-a-innov...,None,None,None,None,None,...,None,Euskadi,50-249,Multisectorial,Multisectorial,Formación,Gratuito,2019-05-15 07:57:37,2025-02-18 12:11:49,None
4,5,"Transferencia de tecnologías, patentes y know-...",Incorpora las mejores prácticas de la industri...,Transferencia de tecnología a través de licenc...,http://www.vicomtech.org/transferencia-de-tecn...,None,None,None,None,None,...,None,Euskadi,+250,Energía y medioambiente,Industria Inteligente,Implantación de tecnologías,Consultar presupuesto,2019-05-15 08:01:07,2022-11-30 16:11:27,None


In [ ]:
import psycopg2
from openai import OpenAI
import os
import time

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise Exception("OPENAI_API_KEY no está configurada")

client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDING_MODEL = "text-embedding-3-large"

try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    # 1. Crear la columna vector si no existe
    cursor.execute("""
        ALTER TABLE soluciones_es
        ADD COLUMN IF NOT EXISTS description_embed vector(3072)
    """)
    conn.commit()
    print("Columna 'description_embed' de tipo vector(3072) creada o ya existe.")

    # 2. Seleccionar los registros que no tienen embedding
    cursor.execute("""
        SELECT id, concat_description
        FROM soluciones_es
    """)
    rows = cursor.fetchall()
    print(f"{len(rows)} registros a vectorizar...")

    for idx, (row_id, concat_description) in enumerate(rows):
        try:
            # Obtener el embedding usando el modelo large de OpenAI
            response = client.embeddings.create(
                input=concat_description,
                model=EMBEDDING_MODEL
            )
            vector = response.data[0].embedding
            # Insertar el vector en la columna correspondiente
            cursor.execute(
                "UPDATE soluciones_es SET description_embed = %s WHERE id = %s",
                (vector, row_id)
            )
            if (idx + 1) % 10 == 0:
                conn.commit()
                print(f"{idx+1}/{len(rows)} registros vectorizados...")
            time.sleep(0.5)  # Para evitar rate limit
        except Exception as e:
            print(f"Error vectorizando id={row_id}: {e}")
            continue
    conn.commit()
    print("Todos los embeddings generados y guardados.")
except Exception as e:
    print(f"Error general: {e}")
    if 'conn' in locals():
        conn.rollback()
finally:
    if 'conn' in locals():
        cursor.close()
        conn.close()



In [9]:
import random

try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    # 1. Crear la columna 'prioridad' si no existe
    cursor.execute("""
        ALTER TABLE soluciones_es
        ADD COLUMN IF NOT EXISTS prioridad INT
    """)
    conn.commit()
    print("Columna 'prioridad' creada o ya existe.")

    # 2. Seleccionar los registros que no tienen valor en 'prioridad'
    cursor.execute("""
        SELECT id FROM soluciones_es
        WHERE prioridad IS NULL
    """)
    ids = cursor.fetchall()
    print(f"{len(ids)} registros a asignar prioridad...")

    for idx, (row_id,) in enumerate(ids):
        # 98% probabilidad de 0, 2% de 1
        prioridad = 1 if random.random() < 0.02 else 0
        cursor.execute(
            "UPDATE soluciones_es SET prioridad = %s WHERE id = %s",
            (prioridad, row_id)
        )
        if (idx + 1) % 20 == 0:
            conn.commit()
            print(f"{idx+1}/{len(ids)} prioridades asignadas...")
    conn.commit()
    print("Prioridades asignadas aleatoriamente a todos los registros.")
except Exception as e:
    print(f"Error asignando prioridades: {e}")
    if 'conn' in locals():
        conn.rollback()


Columna 'prioridad' creada o ya existe.
1666 registros a asignar prioridad...
20/1666 prioridades asignadas...
40/1666 prioridades asignadas...
60/1666 prioridades asignadas...
80/1666 prioridades asignadas...
100/1666 prioridades asignadas...
120/1666 prioridades asignadas...
140/1666 prioridades asignadas...
160/1666 prioridades asignadas...
180/1666 prioridades asignadas...
200/1666 prioridades asignadas...
220/1666 prioridades asignadas...
240/1666 prioridades asignadas...
260/1666 prioridades asignadas...
280/1666 prioridades asignadas...
300/1666 prioridades asignadas...
320/1666 prioridades asignadas...
340/1666 prioridades asignadas...
360/1666 prioridades asignadas...
380/1666 prioridades asignadas...
400/1666 prioridades asignadas...
420/1666 prioridades asignadas...
440/1666 prioridades asignadas...
460/1666 prioridades asignadas...
480/1666 prioridades asignadas...
500/1666 prioridades asignadas...
520/1666 prioridades asignadas...
540/1666 prioridades asignadas...
560/1666